In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StringType, StructType, IntegerType

In [2]:
# spark = SparkSession.builder \
#     .appName("IcebergTableLoad") \
#     .config("spark.master", "spark://spark-master:7077") \
#     .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
#     .config("spark.sql.catalog.spark_catalog.type", "hive") \
#     .config("spark.sql.catalog.spark_catalog.uri", "thrift://hive-metastore:9083") \
#     .config("spark.hadoop.fs.s3a.access.key", "dp-root-user") \
#     .config("spark.hadoop.fs.s3a.secret.key", "dp-root-password") \
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
#     .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#     .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
#     .config("spark.sql.catalogImplementation", "hive") \
#     .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1,org.apache.hadoop:hadoop-aws:3.3.4") \
#     .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
#     .enableHiveSupport() \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("Iceberg with REST Catalog") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.demo.type", "rest") \
    .config("spark.sql.catalog.demo.uri", "http://iceberg-rest:8181") \
    .config("spark.sql.catalog.demo.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.demo.s3.endpoint", "http://minio:9000") \
    .config("spark.sql.catalog.demo.default-namespace", "demo") \
    .config("spark.sql.catalog.demo.s3.path-style-access", "true") \
    .config("spark.hadoop.fs.s3a.access.key", "dp-root-user") \
    .config("spark.hadoop.fs.s3a.secret.key", "dp-root-password") \
    .config("spark.hadoop.fs.s3a.region", "us-west-2") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1,org.apache.hadoop:hadoop-aws:3.3.4,org.apache.iceberg:iceberg-aws-bundle:1.6.1") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()

In [3]:
bucket_name = "warehouse"
file_path = f"s3a://{bucket_name}/test/demo/demo-table"

In [4]:
spark.sql("SHOW TABLES IN demo.demo").show()

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|     demo|demo-table|      false|
+---------+----------+-----------+



In [5]:
spark.sql("SELECT * FROM demo.demo.`demo-table`").show()

+-----------+---+--------+
|       City|Age|    Name|
+-----------+---+--------+
|Los Angeles| 28|Jane Doe|
|Los Angeles| 28|Jane Doe|
+-----------+---+--------+

